In [27]:
# ===========================
# Task-4: Memory-safe notebook
# ===========================

# Cell 0 — imports & settings
import os, json, warnings
import numpy as np, pandas as pd
import scipy.sparse as sp
import matplotlib.pyplot as plt, seaborn as sns
sns.set()
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, SGDRegressor
from sklearn.metrics import (
    mean_squared_error, r2_score, mean_absolute_error,
    roc_auc_score, precision_score, recall_score, f1_score, accuracy_score
)
from sklearn.feature_extraction import FeatureHasher

# Optional: XGBoost + SHAP
try:
    import xgboost as xgb
    HAS_XGB = True
except Exception:
    HAS_XGB = False

try:
    import shap
    HAS_SHAP = True
except Exception:
    HAS_SHAP = False

# Create folders
os.makedirs("reports/interim/figures", exist_ok=True)
os.makedirs("models", exist_ok=True)
print("xgboost:", HAS_XGB, "shap:", HAS_SHAP)


xgboost: True shap: True


In [28]:
# Cell 1 — robust loader (edit paths if needed)
candidates = [
    "../data/raw/merged_dataset.csv",
    "merged_dataset.csv",
    "task3_merged_dataset.csv",
    "data/raw/insurance_data.txt",
    "data.csv"
]
df = None
for p in candidates:
    if os.path.exists(p):
        try:
            df = pd.read_csv(p, low_memory=True)
            print("Loaded:", p, "shape:", df.shape)
            break
        except Exception as e:
            print("Found but could not read:", p, e)

if df is None:
    raise FileNotFoundError("No merged_dataset found. Place merged_dataset.csv in notebook folder.")

# Basic cleaning & numeric conversions
df.columns = df.columns.str.strip()
for col in ["TotalClaims","TotalPremium","CalculatedPremiumPerTerm","CustomValueEstimate"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(r"[^0-9\.\-]", "", regex=True), errors='coerce')

# derive common fields
if "ClaimEvent" not in df.columns and "TotalClaims" in df.columns:
    df["ClaimEvent"] = (df["TotalClaims"].fillna(0).astype(float) > 0).astype(int)
if "Margin" not in df.columns and "TotalPremium" in df.columns and "TotalClaims" in df.columns:
    df["Margin"] = df["TotalPremium"].fillna(0) - df["TotalClaims"].fillna(0)
if "Severity" not in df.columns:
    df["Severity"] = df["TotalClaims"].where(df.get("ClaimEvent",0)==1)

print("Columns:", len(df.columns))


Loaded: ../data/raw/merged_dataset.csv shape: (1000099, 55)
Columns: 55


In [29]:
# Cell 2 — vectorized feature engineering (fast)
d = df.copy()

# TransactionMonth -> year
if "TransactionMonth" in d.columns:
    d["TransactionMonth"] = pd.to_datetime(d["TransactionMonth"], errors="coerce")
    d["transaction_year"] = d["TransactionMonth"].dt.year

# vehicle_age (vectorized)
if "RegistrationYear" in d.columns:
    d["RegistrationYear"] = pd.to_numeric(d["RegistrationYear"], errors="coerce")
    d["vehicle_age"] = d["transaction_year"] - d["RegistrationYear"]

# normalize text columns (lowercase, strip)
text_cols = ["make","Model","VehicleType","Gender","Province","PostalCode","CoverType","Product"]
for c in text_cols:
    if c in d.columns:
        d[c] = d[c].astype(str).str.strip().str.lower().replace("nan", np.nan)

# Candidate features: adjust as needed
candidate_features = [c for c in [
    "vehicle_age","make","VehicleType","Gender","Province","PostalCode",
    "TermFrequency","AlarmImmobiliser","TrackingDevice","CustomValueEstimate",
    "Cylinders","kilowatts","bodytype","NumberOfDoors","NumberOfVehiclesInFleet",
    "SumInsured","RegistrationYear"
] if c in d.columns]

print("Candidate features:", candidate_features)


Candidate features: ['vehicle_age', 'make', 'VehicleType', 'Gender', 'Province', 'PostalCode', 'TermFrequency', 'AlarmImmobiliser', 'TrackingDevice', 'CustomValueEstimate', 'Cylinders', 'kilowatts', 'bodytype', 'NumberOfDoors', 'NumberOfVehiclesInFleet', 'SumInsured', 'RegistrationYear']


In [30]:
# Cell 3 — prepare datasets
freq_df = d.copy()                       # full dataset – classification
severity_df = d[d.get("ClaimEvent",0) == 1].copy()  # only policies with claims

print("freq rows:", freq_df.shape[0], "severity rows:", severity_df.shape[0])


freq rows: 1000099 severity rows: 2788


In [31]:
# -----------------------
# FIXED MEMORY-SAFE PREPROCESSOR
# -----------------------
from sklearn.preprocessing import FunctionTransformer
import scipy.sparse as sp

# numeric vs categorical split
numeric_features = [c for c in candidate_features if freq_df[c].dtype.kind in "biufc"]
categorical_candidates = [c for c in candidate_features if c not in numeric_features]

# cardinality check
cardinalities = {c: freq_df[c].nunique() for c in categorical_candidates}
HASH_THRESHOLD = 100
hash_features = [c for c,v in cardinalities.items() if v > HASH_THRESHOLD]
ohe_features = [c for c in categorical_candidates if c not in hash_features]

print("Numeric:", numeric_features)
print("OneHot (low-card):", ohe_features)
print("Hasher (high-card):", hash_features)

# numeric pipeline
num_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler(with_mean=False)),  # sparse-friendly
    ("to_csr", FunctionTransformer(lambda X: sp.csr_matrix(X), validate=False))
])

# one-hot pipeline
cat_ohe_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=True))
]) if len(ohe_features) > 0 else ("drop_ohe", "drop", [])

# --- FIXED HASHER PIPELINE ---
def df_to_dict_list(X):
    dfX = pd.DataFrame(X, columns=hash_features).astype(str)
    dfX = dfX.replace("nan", "")
    return dfX.to_dict(orient="records")

hash_pipe = Pipeline([
    ("to_dict", FunctionTransformer(df_to_dict_list, validate=False)),
    ("hasher", FeatureHasher(n_features=128, input_type="dict"))
]) if len(hash_features) > 0 else ("drop_hash", "drop", [])

# Build final sparse preprocessor
transformers = []
if len(numeric_features) > 0:
    transformers.append(("num", num_pipe, numeric_features))
if len(ohe_features) > 0:
    transformers.append(("ohe", cat_ohe_pipe, ohe_features))
if len(hash_features) > 0:
    transformers.append(("hash", hash_pipe, hash_features))

preprocessor = ColumnTransformer(transformers, sparse_threshold=1.0)

print("Preprocessor ready. Sections:", [t[0] for t in transformers])


Numeric: ['vehicle_age', 'CustomValueEstimate', 'Cylinders', 'kilowatts', 'NumberOfDoors', 'NumberOfVehiclesInFleet', 'SumInsured', 'RegistrationYear']
OneHot (low-card): ['make', 'VehicleType', 'Gender', 'Province', 'TermFrequency', 'AlarmImmobiliser', 'TrackingDevice', 'bodytype']
Hasher (high-card): ['PostalCode']
Preprocessor ready. Sections: ['num', 'ohe', 'hash']


In [32]:
# Cell 5 — Helper: safe preprocess function (returns scipy.sparse.csr_matrix)
def safe_transform_preprocessor(prep, X_df):
    """
    Fit/transform or transform X_df with preprocessor and ensure sparse CSR output.
    Use fit_transform on training split and transform on others.
    """
    Xt = prep.fit_transform(X_df) if not hasattr(prep, "_is_fitted") or not getattr(prep, "_is_fitted", False) else prep.transform(X_df)
    # Mark as fitted
    setattr(prep, "_is_fitted", True)
    if sp.issparse(Xt):
        return Xt.tocsr()
    else:
        # convert dense to CSR (should be small if only numeric)
        return sp.csr_matrix(Xt)


In [33]:
# Cell 6 — Frequency modeling (sparse, memory-safe, final reviewed version)

from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, precision_score, recall_score,
    f1_score, accuracy_score
)

# -------------------------------
# 0. Optional: check XGBoost availability
# -------------------------------
try:
    import xgboost as xgb
    HAS_XGB = True
except:
    HAS_XGB = False
    print("XGBoost not available — skipping boosted model.")


# -------------------------------
# 1. Extract features and target
# -------------------------------
Xf = freq_df[candidate_features].copy()
yf = freq_df["ClaimEvent"].astype(int)


# -------------------------------
# 2. PROTOTYPE-SAFE SAMPLING (STRATIFIED)
# -------------------------------
MAX_ROWS_PROTOTYPE = 300_000

if Xf.shape[0] > MAX_ROWS_PROTOTYPE:
    print(f"Large dataset detected — stratified sampling to {MAX_ROWS_PROTOTYPE} rows")

    # stratified sample based on target balance
    Xf_sample, _, yf_sample, _ = train_test_split(
        Xf, yf,
        train_size=MAX_ROWS_PROTOTYPE,
        random_state=42,
        stratify=yf
    )
    Xf, yf = Xf_sample, yf_sample


# -------------------------------
# 3. Train/test split BEFORE preprocessing
# -------------------------------
Xf_train_raw, Xf_test_raw, yf_train, yf_test = train_test_split(
    Xf, yf,
    test_size=0.20,
    random_state=42,
    stratify=yf
)


# -------------------------------
# 4. Fit preprocessor (sparse-safe)
# -------------------------------
print("Fitting sparse preprocessor...")
Xf_train = preprocessor.fit_transform(Xf_train_raw)

if not sp.issparse(Xf_train):
    Xf_train = sp.csr_matrix(Xf_train)

Xf_test = preprocessor.transform(Xf_test_raw)
if not sp.issparse(Xf_test):
    Xf_test = sp.csr_matrix(Xf_test)

print("Preprocessed shapes:", Xf_train.shape, Xf_test.shape)


# -------------------------------
# 5. Logistic Regression (sparse-friendly)
# -------------------------------
print("Training Logistic Regression (saga, balanced)...")

log = LogisticRegression(
    solver="saga",
    max_iter=400,
    n_jobs=-1,
    class_weight="balanced"  # improves recall for rare claims
)

log.fit(Xf_train, yf_train)

p_log = log.predict_proba(Xf_test)[:, 1]
pred_log = log.predict(Xf_test)


clf_results = {}
clf_results["logistic"] = {
    "roc_auc": float(roc_auc_score(yf_test, p_log)),
    "precision": float(precision_score(yf_test, pred_log, zero_division=0)),
    "recall": float(recall_score(yf_test, pred_log, zero_division=0)),
    "f1": float(f1_score(yf_test, pred_log, zero_division=0)),
    "accuracy": float(accuracy_score(yf_test, pred_log))
}

print("Logistic Regression Metrics:", clf_results["logistic"])


# -------------------------------
# 6. XGBoost (optional, only if installed)
# -------------------------------
if HAS_XGB:
    print("Training XGBoost on sparse DMatrix...")

    dtrain = xgb.DMatrix(Xf_train, label=yf_train)
    dtest = xgb.DMatrix(Xf_test, label=yf_test)

    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "verbosity": 0,
        "n_jobs": -1
    }

    xgb_model = xgb.train(params, dtrain, num_boost_round=150)

    p_xgb = xgb_model.predict(dtest)
    pred_xgb = (p_xgb >= 0.5).astype(int)

    clf_results["xgb"] = {
        "roc_auc": float(roc_auc_score(yf_test, p_xgb)),
        "precision": float(precision_score(yf_test, pred_xgb, zero_division=0)),
        "recall": float(recall_score(yf_test, pred_xgb, zero_division=0)),
        "f1": float(f1_score(yf_test, pred_xgb, zero_division=0)),
        "accuracy": float(accuracy_score(yf_test, pred_xgb))
    }

    print("XGBoost Metrics:", clf_results["xgb"])


# -------------------------------
# 7. Final result dictionary
# -------------------------------
clf_results


Large dataset detected — stratified sampling to 300000 rows
Fitting sparse preprocessor...
Preprocessed shapes: (240000, 212) (60000, 212)
Training Logistic Regression (saga, balanced)...
Logistic Regression Metrics: {'roc_auc': 0.5585926737603295, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': 0.9972166666666666}
Training XGBoost on sparse DMatrix...
XGBoost Metrics: {'roc_auc': 0.872477497497776, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': 0.9971833333333333}


{'logistic': {'roc_auc': 0.5585926737603295,
  'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'accuracy': 0.9972166666666666},
 'xgb': {'roc_auc': 0.872477497497776,
  'precision': 0.0,
  'recall': 0.0,
  'f1': 0.0,
  'accuracy': 0.9971833333333333}}

In [34]:
# Cell 7 — Severity modeling (regression) memory-safe
severity_metrics = {}

if severity_df.shape[0] < 30:
    print("Not enough claim rows to train severity models robustly.")
else:
    Xs_raw = severity_df[candidate_features].copy()
    ys = severity_df["TotalClaims"].astype(float).fillna(0)

    # --------------------
    # IMPORTANT:
    # Do NOT fit preprocessor again. Only transform.
    # --------------------
    Xs_proc = preprocessor.transform(Xs_raw)

    # For small/medium dataset: attempt densify
    if Xs_proc.shape[0] <= 200_000:
        if sp.issparse(Xs_proc):
            try:
                Xs_dense = Xs_proc.toarray()
            except MemoryError:
                print("Too large to densify — using SGDRegressor on sparse matrix.")
                Xs_dense = None
        else:
            Xs_dense = Xs_proc

        Xs_train, Xs_test, ys_train, ys_test = train_test_split(
            Xs_dense if Xs_dense is not None else Xs_proc,
            ys,
            test_size=0.2,
            random_state=42
        )

        if Xs_dense is not None:
            from sklearn.ensemble import RandomForestRegressor
            rf_reg = RandomForestRegressor(n_estimators=150, n_jobs=-1, random_state=42)
            rf_reg.fit(Xs_train, ys_train)
            p_rf = rf_reg.predict(Xs_test)
            severity_metrics["rf"] = {
                "rmse": float(np.sqrt(mean_squared_error(ys_test, p_rf))),
                "mae": float(mean_absolute_error(ys_test, p_rf)),
                "r2": float(r2_score(ys_test, p_rf))
            }
        else:
            sgd = SGDRegressor(max_iter=1000, tol=1e-3)
            sgd.fit(Xs_train, ys_train)
            p_sgd = sgd.predict(Xs_test)
            severity_metrics["sgd"] = {
                "rmse": float(np.sqrt(mean_squared_error(ys_test, p_sgd))),
                "mae": float(mean_absolute_error(ys_test, p_sgd)),
                "r2": float(r2_score(ys_test, p_sgd))
            }

    else:
        print("Severity too large — sampling 200k for training")

        sample_idx = severity_df.sample(n=200_000, random_state=42).index
        Xs_sample = preprocessor.transform(severity_df.loc[sample_idx, candidate_features])
        ys_sample = ys.loc[sample_idx]

        sgd = SGDRegressor(max_iter=1000, tol=1e-3)
        sgd.fit(Xs_sample, ys_sample)

        test_sample = severity_df.sample(n=20000, random_state=1) if severity_df.shape[0] > 20000 else severity_df
        Xs_test_proc = preprocessor.transform(test_sample[candidate_features])
        p_sgd = sgd.predict(Xs_test_proc)

        severity_metrics["sgd"] = {
            "rmse": float(np.sqrt(mean_squared_error(test_sample["TotalClaims"], p_sgd))),
            "mae": float(mean_absolute_error(test_sample["TotalClaims"], p_sgd)),
            "r2": float(r2_score(test_sample["TotalClaims"], p_sgd))
        }

    print("Severity metrics:", json.dumps(severity_metrics, indent=2))


Severity metrics: {
  "rf": {
    "rmse": 37619.386421611336,
    "mae": 16789.621904379484,
    "r2": 0.12002486915087396
  }
}


In [35]:
# Cell 8 — Risk-based premium and saving predictions
# Choose frequency model scores and severity predictions:
# Use logistic model (log) for p_claim; use rf_reg or sgd for severity predictions if available.

# p_claim for full freq_df (do in chunks if too large)
def predict_proba_chunked(model, preproc, X_df, chunk_size=100000):
    """Return probability vector for X_df using preproc.transform + model.predict_proba in chunks (memory-safe)."""
    n = X_df.shape[0]
    probs = np.zeros(n)
    for start in range(0, n, chunk_size):
        end = min(n, start+chunk_size)
        X_chunk = preproc.transform(X_df.iloc[start:end])
        if sp.issparse(X_chunk):
            Xc = X_chunk
        else:
            Xc = sp.csr_matrix(X_chunk)
        probs[start:end] = model.predict_proba(Xc)[:,1]
    return probs

# p_claim (we trained log on Xf_train etc.)
# If preprocessor is fitted, use preprocessor.transform on the full Xf (do in chunks)
p_claim_all = predict_proba_chunked(log, preprocessor, freq_df[candidate_features], chunk_size=200000)

# severity prediction: depending on chosen severity model
if "rf" in locals():
    # rf_reg expects dense; transform in chunks and densify small chunks
    def predict_sev_rf(rf_model, preproc, X_df, chunk_size=50000):
        n = X_df.shape[0]
        preds = np.zeros(n)
        for start in range(0, n, chunk_size):
            end = min(n, start+chunk_size)
            Xc = preproc.transform(X_df.iloc[start:end])
            if sp.issparse(Xc):
                Xc = Xc.toarray()
            preds[start:end] = rf_model.predict(Xc)
        return preds
    pred_sev_all = predict_sev_rf(rf_reg, preprocessor, freq_df[candidate_features])
elif 'sgd' in locals():
    # sgd accepts sparse
    def predict_sev_sgd(sgd_model, preproc, X_df, chunk_size=200000):
        n = X_df.shape[0]
        preds = np.zeros(n)
        for start in range(0, n, chunk_size):
            end = min(n, start+chunk_size)
            Xc = preproc.transform(X_df.iloc[start:end])
            preds[start:end] = sgd_model.predict(Xc)
        return preds
    pred_sev_all = predict_sev_sgd(sgd, preprocessor, freq_df[candidate_features])
else:
    # fallback: use mean severity
    mean_sev = severity_df["TotalClaims"].mean() if severity_df.shape[0]>0 else 0.0
    pred_sev_all = np.full(freq_df.shape[0], mean_sev)

# Compute risk-based premium
profit_margin = 0.10
expense_loading = 100.0
expected_loss = p_claim_all * pred_sev_all
predicted_premium = expected_loss * (1 + profit_margin) + expense_loading

out = freq_df[["PolicyID"]].copy() if "PolicyID" in freq_df.columns else pd.DataFrame(index=freq_df.index)
out["pred_prob_claim"] = p_claim_all
out["pred_severity"] = pred_sev_all
out["pred_risk_premium"] = predicted_premium
if "CalculatedPremiumPerTerm" in freq_df.columns:
    out["true_premium"] = freq_df["CalculatedPremiumPerTerm"].values

out.to_csv("reports/interim/predicted_premiums_memory_safe.csv", index=False)
print("Saved predicted premiums:", out.shape)


Saved predicted premiums: (1000099, 5)


In [36]:
# === Cell 9: Feature importance + SHAP ===
# build feature names after preprocessor fit
num_names = numeric_features
cat_names = []
try:
    ohe = preprocessor.named_transformers_['cat'].named_steps['ohe']
    cat_names = list(ohe.get_feature_names_out(categorical_features))
except Exception:
    cat_names = []
feature_names = num_names + cat_names

# RF severity importances
try:
    rf_reg = pipe_rf.named_steps['rf']
    imp = rf_reg.feature_importances_
    fi = pd.DataFrame({"feature": feature_names, "importance": imp}).sort_values("importance", ascending=False).head(30)
    fi.to_csv("reports/interim/severity_feature_importance.csv", index=False)
    display(fi.head(10))
except Exception as e:
    print("Severity feature importance error:", e)

# RF frequency importances
try:
    fi2 = pd.DataFrame({"feature": feature_names, "importance": rf.feature_importances_}).sort_values("importance", ascending=False).head(30)
    fi2.to_csv("reports/interim/freq_feature_importance.csv", index=False)
    display(fi2.head(10))
except Exception as e:
    print("Frequency feature importance error:", e)

# SHAP (optional)
if HAS_SHAP:
    try:
        explainer = shap.TreeExplainer(rf_reg)
        # use a sample of test rows transformed
        Xs_sample = preprocessor.transform(Xs_test)
        shap_vals = explainer.shap_values(Xs_sample)
        shap.summary_plot(shap_vals, Xs_sample, feature_names=feature_names, show=False)
        plt.savefig("reports/interim/figures/shap_severity_summary.png", dpi=150, bbox_inches='tight')
        plt.close()
        # record mean abs shap
        mean_abs = np.abs(shap_vals).mean(axis=0)
        shap_df = pd.DataFrame({"feature":feature_names, "mean_abs_shap": mean_abs}).sort_values("mean_abs_shap", ascending=False)
        shap_df.head(10).to_csv("reports/interim/severity_shap_summary.csv", index=False)
        display(shap_df.head(10))
    except Exception as e:
        print("SHAP failed:", e)
else:
    print("SHAP not installed; skip shap explanation.")


Severity feature importance error: name 'pipe_rf' is not defined
Frequency feature importance error: name 'rf' is not defined
SHAP failed: X has 212 features, but ColumnTransformer is expecting 17 features as input.


In [38]:
# === Cell 10: Top-5 human-readable interpretations & per-year effect example (FIXED) ===
# If SHAP summary exists, produce sentences; otherwise use RF importances.

shap_path = "reports/severity_shap_summary.csv"

# ----- SHAP top features -----
if os.path.exists(shap_path):
    shap_t = pd.read_csv(shap_path).head(10)
    for i, row in shap_t.head(5).iterrows():
        print(
            f"- {i + 1}. {row['feature']}: mean|SHAP|={row['mean_abs_shap']:.3f} — "
            f"higher values increase predicted claim amount on average."
        )
else:
    fi_path = "reports/severity_feature_importance.csv"
    if os.path.exists(fi_path):
        fi = pd.read_csv(fi_path).head(10)
        for i, row in fi.iterrows():
            print(f"- {i + 1}. {row['feature']}: importance={row['importance']:.4f}")

# ----- Per-year vehicle_age effect estimation -----
# FIX: use rf_reg (trained model) instead of pipe_rf

if (
    "vehicle_age" in candidate_features
    and "Severity" in severity_df.columns
    and severity_df.shape[0] > 30
):
    try:
        # Prepare raw features
        Xs_raw = severity_df[candidate_features].copy()

        # Preprocess features (rf_reg trained on dense preprocessed data)
        Xs_proc = preprocessor.transform(Xs_raw)

        if sp.issparse(Xs_proc):
            Xs_proc = Xs_proc.toarray()

        # Predict severity
        sev_pred_all = rf_reg.predict(Xs_proc)

        # Build temp df for regression
        tmp = severity_df.copy()
        tmp["sev_pred"] = sev_pred_all
        tmp = tmp[tmp["vehicle_age"].notnull()]

        if tmp.shape[0] > 20:
            coef = np.polyfit(
                tmp["vehicle_age"].astype(float),
                tmp["sev_pred"].astype(float),
                1
            )[0]

            print(
                f"\nEstimate: each additional year of vehicle_age increases "
                f"predicted claim amount by ~{coef:.2f} currency units "
                f"(linear approximation)."
            )

    except NameError:
        print("\nCannot perform vehicle_age effect estimation: Severity model (rf_reg) not defined.")
    except Exception as e:
        print(f"\nCannot perform vehicle_age effect estimation: An error occurred ({e}).")



Estimate: each additional year of vehicle_age increases predicted claim amount by ~-1079.66 currency units (linear approximation).
